# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tepecoacuilco de trujano,18.3000,-99.4833,21.26,74,8,0.55,MX,1731302227
1,1,maralal,1.0968,36.6980,17.35,73,73,3.53,KE,1731302228
2,2,georgetown,5.4112,100.3354,31.96,74,20,3.60,MY,1731302229
3,3,ribeira grande,38.5167,-28.7000,20.99,91,33,9.83,PT,1731302230
4,4,port-aux-francais,-49.3500,70.2167,4.84,88,75,9.52,TF,1731302231


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
weather_con_df = city_data_df.loc[(city_data_df["Max Temp"] < 24)
                                       & (city_data_df["Max Temp"] > 22)]

# Drop any rows with null values
weather_con_df.dropna()
# Display sample data
weather_con_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,hawaiian paradise park,19.5933,-154.9731,22.16,94,64,3.50,US,1731302243
16,16,santa elena de uairen,4.6167,-61.1333,23.49,52,77,0.67,VE,1731302246
58,58,chimore,-16.9833,-65.1333,23.14,97,91,0.29,BO,1731302292
106,106,avarua,-21.2078,-159.7750,23.03,73,40,4.63,CK,1731302345
120,120,tomatlan,19.9333,-105.2500,22.25,97,0,1.00,MX,1731302361


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_con_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

C:\Users\duvoe\AppData\Local\Temp\ipykernel_25368\1947551028.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
13,hawaiian paradise park,US,19.5933,-154.9731,94,
16,santa elena de uairen,VE,4.6167,-61.1333,52,
58,chimore,BO,-16.9833,-65.1333,97,
106,avarua,CK,-21.2078,-159.7750,73,
120,tomatlan,MX,19.9333,-105.2500,97,
125,pauini,BR,-7.7136,-66.9764,96,
126,inirida,CO,3.8653,-67.9239,97,
135,inashiki,JP,35.9563,140.3236,84,
161,port macquarie,AU,-31.4333,152.9167,80,
166,moyale,KE,3.5167,39.0584,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
santa elena de uairen - nearest hotel: Posada Villa Fairmont
chimore - nearest hotel: Aquí me quedo
avarua - nearest hotel: Paradise Inn
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
pauini - nearest hotel: No hotel found
inirida - nearest hotel: Residencias El Turista del Guainia
inashiki - nearest hotel: 湯ったり館 
port macquarie - nearest hotel: The Observatory
moyale - nearest hotel: Paul’s guest house
kirando - nearest hotel: Lakeshore Lodge
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
nassau village-ratliff - nearest hotel: No hotel found
gweru - nearest hotel: Midlands Hotel
holualoa - nearest hotel: Kona Hotel
cabo san lucas - nearest hotel: Comfort Rooms
gebeit - nearest hotel: No hotel found
frontera - nearest hotel: Quirino
seminole - nearest hotel: Best Western Plus Sanford
as sulayyil - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
minas de mataham

,City,Country,Lat,Lng,Humidity,Hotel Name
13,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
16,santa elena de uairen,VE,4.6167,-61.1333,52,Posada Villa Fairmont
58,chimore,BO,-16.9833,-65.1333,97,Aquí me quedo
106,avarua,CK,-21.2078,-159.7750,73,Paradise Inn
120,tomatlan,MX,19.9333,-105.2500,97,Hotel Hacienda Vieja Tomatlan
125,pauini,BR,-7.7136,-66.9764,96,No hotel found
126,inirida,CO,3.8653,-67.9239,97,Residencias El Turista del Guainia
135,inashiki,JP,35.9563,140.3236,84,湯ったり館
161,port macquarie,AU,-31.4333,152.9167,80,The Observatory
166,moyale,KE,3.5167,39.0584,62,Paul’s guest house


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)